### Import Dependencies

In [4]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

from feature_engine.outliers import OutlierTrimmer
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer,
    AddMissingIndicator
)
from feature_engine.encoding import RareLabelEncoder, OrdinalEncoder
from feature_engine.outliers import OutlierTrimmer

### Load Train Data

In [14]:
# DATA_PATH = "data/cardekho_dataset.csv"
DATA_PATH = "cardekho_dataset.csv"

data = pd.read_csv(DATA_PATH)

# basic cleaning
data = data.drop_duplicates()
data = data.drop(columns=["Unnamed: 0"], errors="ignore")


### Apply IQR outlier removal ON FULL DATA

In [15]:
num_cols = data.select_dtypes(include=np.number).columns

iqr_cols = [
    col for col in num_cols
    if data[col].quantile(0.75) - data[col].quantile(0.25) > 0
]

from feature_engine.outliers import OutlierTrimmer

iqr_trimmer = OutlierTrimmer(
    capping_method="iqr",
    tail="both",
    fold=1.5,
    variables=iqr_cols
)

data = iqr_trimmer.fit_transform(data)


### Prepare Train and Test Data

In [16]:
X = data.drop("selling_price", axis=1)
y = np.log(data["selling_price"])   # log transform target

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.1,
    random_state=0
)

### Config

In [17]:
RARE_LABEL_THRESHOLD = 0.01
IQR_FOLD = 1.5
RANDOM_STATE = 42
MODEL_OUTPUT_PATH = "artifacts/final_model_pipeline.joblib"

### Data Pre-Processing Pipeline

In [18]:
# Identify columns
cat_vars = X_train.select_dtypes(include="object").columns.tolist()
num_vars = X_train.select_dtypes(exclude="object").columns.tolist()

# Numerical pipeline
from feature_engine.imputation import MeanMedianImputer, AddMissingIndicator

num_pipeline = Pipeline(steps=[
    ("missing_indicator", AddMissingIndicator(variables=num_vars)),
    ("mean_imputer", MeanMedianImputer(
        imputation_method="mean",
        variables=num_vars
    ))
])

# Categorical pipeline
from feature_engine.imputation import CategoricalImputer
from feature_engine.encoding import RareLabelEncoder, OrdinalEncoder

cat_pipeline = Pipeline(steps=[
    ("missing", CategoricalImputer(
        imputation_method="missing",
        variables=cat_vars
    )),
    ("rare", RareLabelEncoder(
        tol=0.01,
        variables=cat_vars
    )),
    ("ordinal", OrdinalEncoder(
        encoding_method="ordered",
        variables=cat_vars
    ))
])

# ColumnTransformer (now SAFE)
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipeline, num_vars),
        ("cat", cat_pipeline, cat_vars)
    ],
    remainder="drop"
)

# Final Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

final_pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("scaler", MinMaxScaler()),
    ("model", RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ))
])

### Train Model

In [19]:
final_pipeline.fit(X_train, y_train)

C:\Users\shri\AppData\Roaming\Python\Python311\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable seller_type is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
C:\Users\shri\AppData\Roaming\Python\Python311\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable fuel_type is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
C:\Users\shri\AppData\Roaming\Python\Python311\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable transmission_type is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('scaler', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transfor

### Evaluate Model

In [20]:
y_train_pred = final_pipeline.predict(X_train)
y_test_pred = final_pipeline.predict(X_test)

print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
print("Train R2  :", r2_score(y_train, y_train_pred))

print("Test RMSE :", np.sqrt(mean_squared_error(y_test, y_test_pred)))
print("Test R2   :", r2_score(y_test, y_test_pred))


Train RMSE: 0.06858965619299301
Train R2  : 0.9797655492403645
Test RMSE : 0.15848993684401125
Test R2   : 0.8942299337267164


### Persist Model or Pipeline

In [21]:
# joblib.dump(final_pipeline, MODEL_OUTPUT_PATH)
joblib.dump(final_pipeline, r'FINAL_97')
print(f"Model saved at: {MODEL_OUTPUT_PATH}")


Model saved at: artifacts/final_model_pipeline.joblib


### Score New Data

In [22]:
def score_new_data(input_df, model_path=MODEL_OUTPUT_PATH):
    pipeline = joblib.load(model_path)
    log_preds = pipeline.predict(input_df)
    return np.exp(log_preds)   # inverse log


### End of Final Pipeline

In [16]:
cat_pipeline = Pipeline(steps=[
    # Missing values
    ("missing_string", CategoricalImputer(
        imputation_method="missing",
        variables=cat_vars
    )),
    
    # Rare labels (<1%)
    ("rare_labels", RareLabelEncoder(
        tol=0.01,
        n_categories=1,
        variables=cat_vars
    )),
    
    # Target-based ordinal encoding
    ("ordinal", OrdinalEncoder(
        encoding_method="ordered",
        variables=cat_vars
    ))
])


In [17]:
### Numerical pipeline

In [18]:
num_pipeline = Pipeline(steps=[
    # Missing indicators
    ("missing_indicator", AddMissingIndicator(variables=num_vars)),
    
    # Mean imputation
    ("mean_impute", MeanMedianImputer(
        imputation_method="mean",
        variables=num_vars
    ))
])


### Combine pipelines

In [19]:
preprocessor = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_vars),
    ("cat", cat_pipeline, cat_vars)
])


### Scaling pipeline

In [20]:
full_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("scaler", MinMaxScaler())
])


### Fit & transform data

In [21]:
X_train = full_pipeline.fit_transform(X_train, y_train)
X_test = full_pipeline.transform(X_test)


### Save the scaler & pipeline

In [22]:
joblib.dump(full_pipeline, "cardekho_preprocessing_pipeline.joblib")

['cardekho_preprocessing_pipeline.joblib']